In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import numpy as np
import pandas as pd

In [2]:
all_train_data = pd.read_excel("./dataset/tb_train_data.xlsx")
all_train_data

,id,user_id,key_id,plan_id,class_id,frequency,target_load,real_load,is_upload,remark,date_str,create_date,record_id
0,209,1.427560e+18,1.427560e+18,0,2,0,24,18,0,NaN,2021-08-17,1629191736522,17
1,210,1.427560e+18,1.427560e+18,0,2,0,24,28,0,NaN,2021-08-17,1629191737823,17
2,211,1.427560e+18,1.427560e+18,0,2,0,24,26,0,NaN,2021-08-17,1629191739336,17
3,212,1.427560e+18,1.427560e+18,0,2,0,24,21,0,NaN,2021-08-17,1629191740735,17
4,213,1.427560e+18,1.427560e+18,0,2,0,24,24,0,NaN,2021-08-17,1629191742040,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1132579,1.069967e+18,1.626168e+18,1620288221267709952,1,1,19,13,0,NaN,2023-02-16,1676543539723,24993
1048571,1132580,1.069967e+18,1.626168e+18,1620288221267709952,1,1,19,31,0,NaN,2023-02-16,1676543542342,24993
1048572,1132581,1.069967e+18,1.626168e+18,1620288221267709952,1,1,19,17,0,NaN,2023-02-16,1676543545996,24993
1048573,1132582,1.069967e+18,1.626168e+18,1620288221267709952,1,1,19,19,0,NaN,2023-02-16,1676543548832,24993


In [32]:
toc_user_list = pd.read_excel("./dataset/海王toc列表.xlsx")
all_users = pd.read_excel("./dataset/tb_train_user.xlsx")
toc_name = toc_user_list["服务对象"].dropna().drop_duplicates()
user_id_list = []
for name in toc_name:
    user_id = all_users[all_users.name==name].user_id
    if len(user_id)!=0:
        user_id_list.append(user_id.iloc[0])


In [6]:
user_id_list = all_train_data.user_id.drop_duplicates()
new_eavl = pd.read_excel("./dataset/new_evaluation_record.xlsx")
for user_id in user_id_list:
    user_train_record = all_train_data[all_train_data.user_id==user_id]
    user_eval_record = new_eavl[new_eavl.user_id==user_id]
    if len(user_train_record)>2000:
      print(user_train_record)
      print(user_eval_record)

          id       user_id        key_id              plan_id  class_id  \
6584    8867  1.481135e+18  1.482237e+18  1482236852850741248         1   
6585    8868  1.481135e+18  1.482237e+18  1482236852850741248         1   
6586    8869  1.481135e+18  1.482237e+18  1482236852850741248         1   
6587    8870  1.481135e+18  1.482237e+18  1482236852850741248         1   
6588    8871  1.481135e+18  1.482237e+18  1482236852850741248         1   
...      ...           ...           ...                  ...       ...   
79346  81848  1.481135e+18  1.527846e+18                    0         0   
79347  81849  1.481135e+18  1.527846e+18                    0         0   
79348  81850  1.481135e+18  1.527846e+18                    0         0   
79349  81851  1.481135e+18  1.527846e+18                    0         0   
79350  81852  1.481135e+18  1.527846e+18                    0         0   

       frequency  target_load  real_load  is_upload  remark    date_str  \
6584           0        

KeyboardInterrupt: 

In [10]:
data_dir = "../../Multidimensional-time-series-with-transformer-main/0307data"
for file in os.listdir(data_dir):
    folder = os.path.join(data_dir,file)
    if os.path.isdir(folder):
        train_data = pd.read_excel(os.path.join(folder,"select_train.xlsx"))
        if len(train_data)>10000:
            train_data.drop(["id","user_id","key_id","plan_id","class_id",
                             "frequency","is_upload","remark","create_date",
                             "record_id"],axis=1,inplace=True)
            train_data = train_data.reindex(columns=["date_str","real_load","target_load"])
            train_data = train_data.rename(columns={"date_str":"date","real_load":"real_load",
                                                    "target_load":"OT"})
            train_data.to_csv(os.path.join("./dataset/wb",file+".csv"),index=False)

In [ ]:
data_dir = "../../Multidimensional-time-series-with-transformer-main/0307data"
drop = [
"310103194908240027",
"342623198107226821",
"310104196506112810",
"450821201009125820"]
for file in os.listdir(data_dir):
    folder = os.path.join(data_dir,file)
    # if file != '34242319790406396X':continue
    if os.path.isdir(folder):
        
        train_data = pd.read_excel(os.path.join(folder,"select_train.xlsx"))
        eval_data = pd.read_excel(os.path.join(folder,"evaluation_records.xlsx"))
        if len(train_data)>2000:
            if file not in drop:
                train_data['real_load'] = savgol_filter(train_data['real_load'],51,2)
                train_data['date'] = pd.to_datetime(train_data['date_str'])
                train_data.set_index('date', inplace=True)

                grouped = train_data.groupby(train_data.index.date)
                mean = grouped.mean(numeric_only=True)
                
                mean = grouped.mean(numeric_only=True)
                mean['bias']=mean.real_load-mean.target_load
                # std = grouped.std()
                # 计算每个日期与第一个日期之间的天数差
                helper = pd.DataFrame({'date': pd.date_range(start=mean.index[0], end=mean.index[-1])})
                
                mean['days'] = (mean.index - mean.index[0]).days
                xmin = mean['days'].min()
                mean['days'] = mean['days'] - xmin

                mean.set_index('days', inplace=True)
                d = pd.concat([helper,mean.real_load,mean.target_load],axis=1)
                d['real_load'] = d['real_load'].interpolate(method='linear')
                d['OT'] = d['target_load'].interpolate(method='linear')
                d.drop(['target_load'],axis=1,inplace=True)
                
                eval_list = []
                vas_list = []
                eval_data.drop(["is_upload"],axis=1,inplace=True)
                eval_records = eval_data.iloc[:,7:12]
                eval_records = np.array(eval_records)
                for x in range(eval_records.shape[0]):
                    sorted_index = np.argsort(eval_records[x,:])
                    top_k_index = sorted_index[::-1][:3]
                    eval_list.append(np.mean(eval_records[x,:][top_k_index]))
                    vas_list.append(1)
                dlen = len(d)
                eval_df = pd.DataFrame(eval_list)
                vas_df = pd.DataFrame(vas_list)
                if len(d)/len(eval_df)<6:
                    eval_df.index = 3*eval_df.index
                    vas_df.index = 3*vas_df.index
                    maxlen = int(dlen/3)
                else:
                    eval_df.index = 6*eval_df.index
                    vas_df.index = 6*vas_df.index
                    maxlen = int(dlen/6)
                eval_df = eval_df.iloc[:maxlen,:]
                eval_df.columns = ["eval"]
                vas_df = vas_df.iloc[:maxlen,:]
                vas_df.columns = ["VAS"]

                d = pd.concat([d,eval_df["eval"],vas_df["VAS"]],axis=1)
                d.fillna(0,inplace=True)
                d = d.reindex(columns=["date","eval","VAS","real_load","OT"])

                if len(d)>30:
                    # d.to_csv(os.path.join("./dataset/wb_days_mv",file+".csv"),index=False)

                    # 将 x 轴的原点设置为 0
                    # print(len(d))

                    plt.figure(figsize=(8,6))
                    plt.plot(d.index, d['real_load'],'--', color='red', linewidth=2,label='load')
                    plt.plot(eval_df.index, eval_df.iloc[:,0],'v', color='green', linewidth=2,label='eval')
                    plt.plot(vas_df.index, vas_df.iloc[:,0],'o', color='purple', linewidth=2,label='VAS')
                    plt.plot(d.index,d['OT'],'--', label='plan')

                    plt.legend()

                    # plt.title('Weight-bearing Data\n'+file)
                    plt.title('Weight-bearing Data')
                    plt.xlabel('Days')
                    plt.ylabel('Load (kg)')

                    plt.show()


In [12]:
drop = [
"310103194908240027",
"342623198107226821",
"310104196506112810",
"450821201009125820"]
user_id_list = all_train_data.user_id.drop_duplicates()
for user_id in user_id_list:
    user_train_record = all_train_data[all_train_data.user_id==user_id]

    if len(user_train_record)>2000:
        train_data = user_train_record
            # if file not in drop:
        train_data['real_load'] = savgol_filter(train_data['real_load'],51,2)
        train_data['date'] = pd.to_datetime(train_data['date_str'])
        train_data.set_index('date', inplace=True)

        grouped = train_data.groupby(train_data.index.date)
        mean = grouped.mean(numeric_only=True)
        
        mean = grouped.mean(numeric_only=True)
        mean['bias']=mean.real_load-mean.target_load
        # std = grouped.std()
        # 计算每个日期与第一个日期之间的天数差
        helper = pd.DataFrame({'date': pd.date_range(start=mean.index[0], end=mean.index[-1])})
        
        mean['days'] = (mean.index - mean.index[0]).days
        xmin = mean['days'].min()
        mean['days'] = mean['days'] - xmin

        mean.set_index('days', inplace=True)
        d = pd.concat([helper,mean.real_load,mean.target_load],axis=1)
        d['real_load'] = d['real_load'].interpolate(method='linear')
        d['OT'] = d['target_load'].interpolate(method='linear')
        d.drop(['target_load'],axis=1,inplace=True)

        print(d)
        # if len(d)>30:


        # plt.figure(figsize=(8,6))
        # plt.plot(d.index, d['real_load'],'--', color='red', linewidth=2,label='load')
        # plt.plot(d.index,d['OT'],'--', label='plan')

        # plt.legend()

        # # plt.title('Weight-bearing Data\n'+file)
        # plt.title('Weight-bearing Data')
        # plt.xlabel('Days')
        # plt.ylabel('Load (kg)')

        # plt.show()

/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['real_load'] = savgol_filter(train_data['real_load'],51,2)
/tmp/ipykernel_14884/1300648724.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['date'] = pd.to_datetime(train_data['date_str'])
/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

          date  real_load         OT
0   2022-01-15  15.091114  16.058824
1   2022-01-16  16.059866  16.785862
2   2022-01-17  17.028619  17.512900
3   2022-01-18  17.997371  18.239938
4   2022-01-19  18.966123  18.966976
..         ...        ...        ...
122 2022-05-17  50.577614  50.000000
123 2022-05-18  50.622971  50.000000
124 2022-05-19  50.668328  50.000000
125 2022-05-20  50.713685  50.000000
126 2022-05-21  51.431756  50.000000

[127 rows x 3 columns]
          date  real_load         OT
0   2022-01-18  16.957400   6.000000
1   2022-01-19  17.097875   6.156614
2   2022-01-20  17.238350   6.313228
3   2022-01-21  17.378825   6.469841
4   2022-01-22  17.519300   6.626455
..         ...        ...        ...
138 2022-06-05  53.301213  48.000000
139 2022-06-06  53.356141  48.000000
140 2022-06-07  53.411070  48.000000
141 2022-06-08  53.465998  48.000000
142 2022-06-09  53.520927  48.000000

[143 rows x 3 columns]


/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['real_load'] = savgol_filter(train_data['real_load'],51,2)
/tmp/ipykernel_14884/1300648724.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['date'] = pd.to_datetime(train_data['date_str'])
/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

         date  real_load         OT
0  2022-03-05  19.280258  20.244444
1  2022-03-06  20.517398  22.043796
2  2022-03-07  10.125430  10.000000
3  2022-03-08  10.503186  10.000000
4  2022-03-09  11.978428  12.000000
..        ...        ...        ...
91 2022-06-04  52.517103  48.000000
92 2022-06-05  52.308109  49.000000
93 2022-06-06  52.529245  49.669421
94 2022-06-07  52.788814  49.000000
95 2022-06-08  52.471259  49.000000

[96 rows x 3 columns]
         date  real_load         OT
0  2022-04-14  54.021759  52.692308
1  2022-04-15  57.321429  60.333333
2  2022-04-16  62.987235  62.000000
3  2022-04-17  66.549399  70.000000
4  2022-04-18  67.228392  71.033679
..        ...        ...        ...
75 2022-06-28  71.693060  56.503297
76 2022-06-29  70.711763  55.358242
77 2022-06-30  69.730467  54.213187
78 2022-07-01  68.749171  53.068132
79 2022-07-02  67.767874  51.923077

[80 rows x 3 columns]


/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['real_load'] = savgol_filter(train_data['real_load'],51,2)
/tmp/ipykernel_14884/1300648724.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['date'] = pd.to_datetime(train_data['date_str'])
/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

...

         date  real_load         OT
0  2022-07-20   4.775678   6.000000
1  2022-07-21   5.040512   6.000000
2  2022-07-22   5.305346   6.000000
3  2022-07-23   5.570181   6.000000
4  2022-07-24   5.972297   6.000000
5  2022-07-25   6.078661   6.000000
6  2022-07-26   6.496399   7.000000
7  2022-07-27   6.871081   7.000000
8  2022-07-28   7.053039   7.000000
9  2022-07-29   7.165487   7.000000
10 2022-07-30   7.245913   7.000000
11 2022-07-31   7.341658   7.000000
12 2022-08-01   7.704721   7.000000
13 2022-08-02   8.528122   8.000000
14 2022-08-03   8.567090   8.000000
15 2022-08-04   8.995055   8.000000
16 2022-08-05   8.596827   8.000000
17 2022-08-06   8.905119   8.500000
18 2022-08-07   9.581419   9.000000
19 2022-08-08  10.032951   9.000000
20 2022-08-09   9.842014   9.000000
21 2022-08-10  10.918258   9.000000
22 2022-08-11  11.599865  10.000000
23 2022-08-12  12.495472  10.000000
24 2022-08-13  13.609166  10.000000
25 2022-08-14  15.708852  10.000000
26 2022-08-15  16.177135  10

/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['real_load'] = savgol_filter(train_data['real_load'],51,2)
/tmp/ipykernel_14884/1300648724.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['date'] = pd.to_datetime(train_data['date_str'])
/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

         date  real_load         OT
0  2022-08-08  33.544444  33.000000
1  2022-08-09  30.186795  28.956522
2  2022-08-10  28.476537  28.000000
3  2022-08-11  28.558488  28.000000
4  2022-08-12  29.823439  29.000000
5  2022-08-13  30.667481  30.000000
6  2022-08-14  34.359594  32.000000
7  2022-08-15  35.907546  33.000000
8  2022-08-16  37.570589  35.000000
9  2022-08-17  38.744365  36.000000
10 2022-08-18  40.561365  38.000000
11 2022-08-19  41.064758  39.000000
12 2022-08-20  42.854383  40.000000
13 2022-08-21  43.664459  40.000000
14 2022-08-22  43.783320  41.000000
15 2022-08-23  44.295358  41.000000
16 2022-08-24  44.296472  42.000000
17 2022-08-25  45.031711  42.000000
18 2022-08-26  44.360759  43.000000
19 2022-08-27  46.389987  44.000000
20 2022-08-28  48.496286  44.000000
21 2022-08-29  48.944774  45.000000
22 2022-08-30  50.487701  45.000000
23 2022-08-31  51.049020  46.000000
24 2022-09-01  51.624025  46.000000
25 2022-09-02  53.033189  47.000000
26 2022-09-03  53.849739  48

/tmp/ipykernel_14884/1300648724.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['real_load'] = savgol_filter(train_data['real_load'],51,2)
/tmp/ipykernel_14884/1300648724.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['date'] = pd.to_datetime(train_data['date_str'])


KeyboardInterrupt: 